In [3]:
import yfinance as yf
import sqlite3
import pandas as pd
import time

# Path to the existing SQLite database file
DATABASE_PATH = "./stocks.db"

# Nifty 50 Stocks with their Sectors (manually mapped)
nifty_50_stocks = {
    "RELIANCE.NS": "Energy",
    "TCS.NS": "IT",
    "INFY.NS": "IT",
    "HDFCBANK.NS": "Banking",
    "ICICIBANK.NS": "Banking",
    "HINDUNILVR.NS": "FMCG",
    "ITC.NS": "FMCG",
    "SBIN.NS": "Banking",
    "BHARTIARTL.NS": "Telecom",
    "KOTAKBANK.NS": "Banking",
    "LT.NS": "Infrastructure",
    "HCLTECH.NS": "IT",
    "AXISBANK.NS": "Banking",
    "ASIANPAINT.NS": "Paints",
    "MARUTI.NS": "Automobile",
    "SUNPHARMA.NS": "Pharma",
    "TITAN.NS": "Retail",
    "BAJFINANCE.NS": "Financial Services",
    "WIPRO.NS": "IT",
    "ONGC.NS": "Energy",
    "M&M.NS": "Automobile",
    "ULTRACEMCO.NS": "Cement",
    "POWERGRID.NS": "Power",
    "NTPC.NS": "Power",
    "JSWSTEEL.NS": "Steel",
    "TATASTEEL.NS": "Steel",
    "HINDALCO.NS": "Metal",
    "COALINDIA.NS": "Energy",
    "TECHM.NS": "IT",
    "BPCL.NS": "Energy",
    "INDUSINDBK.NS": "Banking",
    "IOC.NS": "Energy",
    "ADANIENT.NS": "Infrastructure",
    "EICHERMOT.NS": "Automobile",
    "GRASIM.NS": "Cement",
    "HDFCLIFE.NS": "Insurance",
    "HEROMOTOCO.NS": "Automobile",
    "UPL.NS": "Agrochemicals",
    "APOLLOHOSP.NS": "Healthcare",
    "BAJAJFINSV.NS": "Financial Services",
    "DIVISLAB.NS": "Pharma",
    "CIPLA.NS": "Pharma",
    "BRITANNIA.NS": "FMCG",
    "NESTLEIND.NS": "FMCG",
    "SBILIFE.NS": "Insurance",
    "TATAMOTORS.NS": "Automobile",
    "ADANIPORTS.NS": "Infrastructure",
    "DRREDDY.NS": "Pharma",
    "BAJAJ-AUTO.NS": "Automobile",
    "SHREECEM.NS": "Cement",
    "LTIM.NS": "IT"
}

# Function to fetch stock data and store it in SQLite
def fetch_and_store_stock_data():
    # Connect to the existing database
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()

    for ticker, sector in nifty_50_stocks.items():
        print(f"Fetching data for {ticker}...")

        # Fetch stock data from Yahoo Finance
        stock = yf.Ticker(ticker)
        try:
            df = stock.history(period="max", start="2015-01-01")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            continue

        if df.empty:
            print(f"No data available for {ticker}")
            continue

        # Add additional columns: Ticker, Market Cap, and Sector
        df["Ticker"] = ticker
        df["Sector"] = sector

        # Fetch Market Cap from Yahoo Finance (sometimes it's missing)
        try:
            market_cap = stock.info.get("marketCap", None)
            df["Marketcap"] = market_cap
        except Exception as e:
            print(f"Error fetching market cap for {ticker}: {e}")
            df["Marketcap"] = None  # Assign None if market cap isn't available

        # Reset index to get Date as a column
        df.reset_index(inplace=True)

        # Select only required columns and rename for consistency
        df = df[["Date", "Ticker", "Open", "High", "Low", "Close", "Volume", "Marketcap", "Sector"]]

        # Convert 'Date' column to string format for database insertion
        df["Date"] = df["Date"].astype(str)

        # Insert data into the existing SQLite table
        df.to_sql("stock_data", conn, if_exists="append", index=False)

        print(f"Data for {ticker} stored successfully.")

        # Prevent hitting API limits
        time.sleep(1)

    # Close database connection
    conn.close()
    print("✅ All stock data stored in SQLite database.")

# Run the function
fetch_and_store_stock_data()


Fetching data for RELIANCE.NS...
Data for RELIANCE.NS stored successfully.
Fetching data for TCS.NS...
Data for TCS.NS stored successfully.
Fetching data for INFY.NS...
Data for INFY.NS stored successfully.
Fetching data for HDFCBANK.NS...
Data for HDFCBANK.NS stored successfully.
Fetching data for ICICIBANK.NS...
Data for ICICIBANK.NS stored successfully.
Fetching data for HINDUNILVR.NS...
Data for HINDUNILVR.NS stored successfully.
Fetching data for ITC.NS...
Data for ITC.NS stored successfully.
Fetching data for SBIN.NS...
Data for SBIN.NS stored successfully.
Fetching data for BHARTIARTL.NS...
Data for BHARTIARTL.NS stored successfully.
Fetching data for KOTAKBANK.NS...
Data for KOTAKBANK.NS stored successfully.
Fetching data for LT.NS...
Data for LT.NS stored successfully.
Fetching data for HCLTECH.NS...
Data for HCLTECH.NS stored successfully.
Fetching data for AXISBANK.NS...
Data for AXISBANK.NS stored successfully.
Fetching data for ASIANPAINT.NS...
Data for ASIANPAINT.NS store

In [1]:
import requests
import pandas as pd
import time

# API Key
NEWS_API_KEY = "818c81d17eef445c8a8434e4d97032da"
NEWS_API_URL = "https://newsapi.org/v2/everything"

# Nifty 50 Stocks and Sectors
nifty_50_stocks = { 
    "RELIANCE.NS": "Energy", "TCS.NS": "IT", "INFY.NS": "IT", "HDFCBANK.NS": "Banking",
    "ICICIBANK.NS": "Banking", "HINDUNILVR.NS": "FMCG", "ITC.NS": "FMCG", "SBIN.NS": "Banking",
    "BHARTIARTL.NS": "Telecom", "KOTAKBANK.NS": "Banking", "LT.NS": "Infrastructure",
    "HCLTECH.NS": "IT", "AXISBANK.NS": "Banking", "ASIANPAINT.NS": "Paints",
    "MARUTI.NS": "Automobile", "SUNPHARMA.NS": "Pharma", "TITAN.NS": "Retail",
    "BAJFINANCE.NS": "Financial Services", "WIPRO.NS": "IT", "ONGC.NS": "Energy",
    "M&M.NS": "Automobile", "ULTRACEMCO.NS": "Cement", "POWERGRID.NS": "Power",
    "NTPC.NS": "Power", "JSWSTEEL.NS": "Steel", "TATASTEEL.NS": "Steel",
    "HINDALCO.NS": "Metal", "COALINDIA.NS": "Energy", "TECHM.NS": "IT", "BPCL.NS": "Energy",
    "INDUSINDBK.NS": "Banking", "IOC.NS": "Energy", "ADANIENT.NS": "Infrastructure",
    "EICHERMOT.NS": "Automobile", "GRASIM.NS": "Cement", "HDFCLIFE.NS": "Insurance",
    "HEROMOTOCO.NS": "Automobile", "UPL.NS": "Agrochemicals", "APOLLOHOSP.NS": "Healthcare",
    "BAJAJFINSV.NS": "Financial Services", "DIVISLAB.NS": "Pharma", "CIPLA.NS": "Pharma",
    "BRITANNIA.NS": "FMCG", "NESTLEIND.NS": "FMCG", "SBILIFE.NS": "Insurance",
    "TATAMOTORS.NS": "Automobile", "ADANIPORTS.NS": "Infrastructure", "DRREDDY.NS": "Pharma",
    "BAJAJ-AUTO.NS": "Automobile", "SHREECEM.NS": "Cement", "LTIM.NS": "IT"
}

# Function to fetch news
def fetch_news(stock_name, sector):
    base_stock_name = stock_name.split('.')[0]
    params = {
        "q": base_stock_name,  # Search for stock name
        "apiKey": NEWS_API_KEY,
        "language": "en",
        "sortBy": "relevancy",
        "pageSize": 50  # Max articles per request
    }

    response = requests.get(NEWS_API_URL, params=params)

    if response.status_code == 200:
        articles = response.json().get("articles", [])
        news_data = []

        for article in articles:
            news_data.append({
                "Stock": stock_name,
                "Sector": sector,
                "Date": article.get("publishedAt", "N/A"),
                "Title": article.get("title", "N/A"),
                "Description": article.get("description", "N/A"),
                "Content": article.get("content", "N/A"),
                "Source": article["source"].get("name", "N/A"),
                "URL": article.get("url", "N/A")
            })

        return news_data
    else:
        print(f"Error fetching news for {stock_name}: {response.status_code} - {response.text}")
        return []

# Collect news for all stocks
all_news = []

for stock, sector in nifty_50_stocks.items():
    print(f"Fetching news for {stock} ({sector})...")
    news = fetch_news(stock, sector)
    all_news.extend(news)
    time.sleep(2)  # Prevent hitting API rate limits

# Save to CSV
df = pd.DataFrame(all_news)
df.to_csv("nifty50_news.csv", index=False, encoding="utf-8")
print("✅ News data saved to nifty50_news.csv")


Fetching news for RELIANCE.NS (Energy)...
Error fetching news for RELIANCE.NS: 429 - {"status":"error","code":"rateLimited","message":"You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests."}
Fetching news for TCS.NS (IT)...
Error fetching news for TCS.NS: 429 - {"status":"error","code":"rateLimited","message":"You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests."}
Fetching news for INFY.NS (IT)...
Error fetching news for INFY.NS: 429 - {"status":"error","code":"rateLimited","message":"You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more req